### This model is based on the T5-base model. We used "transfer learning" to get our model to generate paraphrases as well as ChatGPT. Now we can say that this is one of the best paraphrases of the Hugging Face. The model was trained on 5 epochs and works well with small sentences and questions, we are going to improve its results on big sentences.

#### This model was trained on our [ChatGPT paraphrase dataset](https://huggingface.co/datasets/humarin/chatgpt-paraphrases).

#### This dataset is based on the [Quora paraphrase question](https://www.kaggle.com/competitions/quora-question-pairs), texts from the [SQUAD 2.0](https://huggingface.co/datasets/squad_v2) and the [CNN news dataset](https://huggingface.co/datasets/cnn_dailymail).

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.auto import tqdm  # Use tqdm.auto for compatibility
import torch
import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# device = "cpu"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")

model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [6]:
def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=1,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=512
):
 
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to(device)
 
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )
 
    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return res

In [7]:
data=pd.read_csv('/kaggle/input/watermarked-and-unwatermarked-text-truncated/data_trunk.csv')
data=data.loc[data['label'] == 'watermarked']
data = data.drop('label', axis=1)
data.head()

,Generated Text
1,Some companies failed to pay their workers dur...
3,"Pneumonia can be caused by viruses, bacteria a..."
6,Leaders have taken this issue to bigger office...
8,Bodaboda riders through their association can ...
10,The government has the power to influence acti...


In [8]:
paraphrases_df = pd.DataFrame(columns=["Generated Text", "paraphrased"])
# Use tqdm to show progress
for index, row in tqdm(data.iterrows(), total=len(data)):
    text = row["Generated Text"]  
    paraphrases = paraphrase(text)
#     print(f"Original: {text}")
#     print(f"Paraphrases: {paraphrases[0]}")
    # Create a DataFrame row with original question and paraphrases
    paraphrase_row = pd.DataFrame({"Generated Text": [text], "paraphrased": paraphrases[0]})
    # Append the paraphrase row to the paraphrases DataFrame
    paraphrases_df = pd.concat([paraphrases_df, paraphrase_row], ignore_index=True)
paraphrases_df.to_csv("paraphased-text.csv", index=False)


  0%|          | 0/1000 [00:00<?, ?it/s]

NameError: name 'paraphrases_df' is not defined

In [7]:
# for index, row in data.iterrows():
#     # Assuming 'text' is the column containing the sentences
#     print(row)
#     sentence = row['Generated Text']
#     # Process the sentence
#     output = paraphrase(sentence)
#     df = pd.DataFrame(output)
#     # Save the DataFrame to a CSV file
#     df.to_csv('my_dataset.csv', index=False)

